#### Hi there ! I'm Napadol Thongthavorn (New). I'm a Junior Data Scientist but I have no work experience so I decided to make this portfolio to prove my entry-level skill.
This first project that I'm gonna do is from Barış Karaman's articles "DATA DRIVEN GROWTH WITH PYTHON". It's a series of articles that was designed to explain how to use Python in a simplistic way with combination of programming, data analysis and machine learning. So I'll follow his steps (but not all of his) and try to explain it by my own language and knowledge.

# Online Retail Data

The data set that I'll use here is a sample dataset of an online retail. It's a dataset that contains all purchases made for an online retail company based in the UK during an 8 months period. You can find more information from https://www.kaggle.com/vijayuv/onlineretail and All I'll do is some basic Exploratory Data Analysis to see some basic insights from this dataset. Let's Begin ! :)

In [19]:
#Let's start w/ import libraries
from __future__ import division
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()

In [5]:
data = pd.read_csv('data.csv', encoding = "ISO-8859-1")
#I added parameter encoding because we may get UnicodeDecodeError: 'utf-8' 

In [6]:
# so let's see what we've got from this data
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### We're gonna focus on these following columns :
- Customer ID
- Unit Price
- Quantity
- Invoice Date

### First let find Revenue by this formula
Revenue = sum(UnitPirce * Quantity)

In [7]:
#convert the type of 'InvoiceDate' column from string to datetime.
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [8]:
#creating 'YearMonth' column for reporting and visualization
data['InvoiceYearMonth'] = data['InvoiceDate'].map(lambda date: 100*date.year + date.month)

#### Why it's gotta be 100*date.year + date.month ?
For example if we plus it by indexing data['InvoiceDate'][0].year + data['InvoiceDate'][0].month
we'll get "2022"  (from 2010 + 12) as an answer and we don't want that because we won't know what's year and month is.
If we added 100*date.year + date.month we'll get 201012 (from (100*2010) + 12) so now we know what year and month is ! (or you can use dt.to_period if you prefer)

In [12]:
#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
data['Revenue'] = data['UnitPrice'] * data['Quantity']
revenue_data = data.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
revenue_data

,InvoiceYearMonth,Revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121
5,201105,723333.510
6,201106,691123.120
7,201107,681300.111
8,201108,682680.510
9,201109,1019687.622


### Next, Let's do some visualization from this table !

In [20]:
#X and Y axis inputs for Plotly graph. We use Scatter for line graphs
plot_data = [go.Scatter(x=revenue_data['InvoiceYearMonth'], y=revenue_data['Revenue'],)]
plot_layout = go.Layout(xaxis={"type": "category"},title='Montly Revenue')

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

As we see from this figure. You can notice that the revenue is increased so much from 2011-08 and at its peak in 2011-11 but in 2011-12 it's obviously decreased. So strange right ?
#### So let's have a look at our dataset !

In [14]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,201112,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,201112,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,201112,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,201112,16.60
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,201112,14.85


Can you find out something in 'InvoiceDate' column ?
Yeah right ! The data in December is incomplete. So we've solved some riddle. Next, We'll figure out that what's our Monthly Revenue Growth Rate.

In [15]:
#using pct_change() function to see monthly percentage change and create a new column
revenue_data['MonthlyGrowth'] = revenue_data['Revenue'].pct_change()

#and let's see what It look like
revenue_data

,InvoiceYearMonth,Revenue,MonthlyGrowth
0,201012,748957.020,NaN
1,201101,560000.260,-0.252293
2,201102,498062.650,-0.110603
3,201103,683267.080,0.371850
4,201104,493207.121,-0.278163
5,201105,723333.510,0.466592
6,201106,691123.120,-0.044530
7,201107,681300.111,-0.014213
8,201108,682680.510,0.002026
9,201109,1019687.622,0.493653


we got NaN in our first month of data and that's because we can't get a percent return on the very first day, there's nothing in the past to compare. and that's it !

In [21]:
#again, let's do some visualization by using line graph

#Don't forget to filter 2011-12 out because it's incomplete!
plot_data = [go.Scatter(x=revenue_data.query("InvoiceYearMonth < 201112")['InvoiceYearMonth'],
                        y=revenue_data.query("InvoiceYearMonth < 201112")['MonthlyGrowth'],)]

plot_layout = go.Layout(xaxis={"type": "category"},title='Montly Growth Rate')

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

We can something strange again ! Our growth rate dropped so much from 2011-03 to 2011-04.  What happened ? Maybe due to less active customers, our customers did less orders or they really bought cheaper products ? We can't guess it yet w/o doing a deep-dive analysis.

Next I'm gonna talk about
### Monthly Active Customers
by follow the steps we exactly did for Monthly Revenue but focusing on UK data only (because it has the most records).
We can get the monthly active customers by counting unique CustomerIDs.

In [22]:
#creating a new dataframe focusing only UK
uk_data = data.query("Country=='United Kingdom'").reset_index(drop=True)

#creating monthly active customers dataframe by counting unique Customer IDs
uk_monthly_active = uk_data.groupby('InvoiceYearMonth')['CustomerID'].nunique().reset_index()

#Let take a look at it
uk_monthly_active

,InvoiceYearMonth,CustomerID
0,201012,871
1,201101,684
2,201102,714
3,201103,923
4,201104,817
5,201105,985
6,201106,943
7,201107,899
8,201108,867
9,201109,1177


Remember that in 2011-04 our growth rate is obviously decreased and If we take a look at Monthly Active customers we'll notice that it's dropped from 923 to 817. Let's me do some math real quick (817/923)-1 = -0.1148
#### IT'S dropped around 11.5% !

In [23]:
#And a visualization for better perspective !
plot_data = [go.Bar(x=uk_monthly_active['InvoiceYearMonth'],y=uk_monthly_active['CustomerID'],)]

plot_layout = go.Layout(xaxis={"type": "category"},title='Monthly Active Customers')

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### To figure it out more we need to see the trend for number that our customers have ordered as well.

In [24]:
#create a new dataframe for number of order by using quantity field
monthly_sales_data = uk_data.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()

#now let's see what we got
monthly_sales_data

,InvoiceYearMonth,Quantity
0,201012,298101
1,201101,237381
2,201102,225641
3,201103,279843
4,201104,257666
5,201105,306452
6,201106,258522
7,201107,324129
8,201108,319804
9,201109,458490


In [27]:
#plot
plot_data = [go.Bar(x=monthly_sales_data['InvoiceYearMonth'],y=monthly_sales_data['Quantity'],)]

plot_layout = go.Layout(xaxis={"type": "category"},title='Monthly Total Number of Orders')

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Again, Let's focus on our riddle first. Monthly Total Number of Orders is also decreased in 2011-04 (279k to 257k it's about -8%).
Next, Let's check out our
### Average Revenue per Order

by find our Mean Revenue for every month.

In [29]:
#create a new dataframe for average revenue by taking the mean of it
monthly_order_avg = uk_data.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

#let's see
monthly_order_avg

,InvoiceYearMonth,Revenue
0,201012,16.865860
1,201101,13.614680
2,201102,16.093027
3,201103,16.716166
4,201104,15.773380
5,201105,17.713823
6,201106,16.714748
7,201107,15.723497
8,201108,17.315899
9,201109,18.931723


In [33]:
#plot the bar chart
plot_data = [go.Bar(x=monthly_order_avg['InvoiceYearMonth'],y=monthly_order_avg['Revenue'])]

plot_layout = go.Layout(xaxis={"type": "category"},title='Monthly Order Average')

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Again, monthly order average also dropped in 2011-04 (16.7 to 15.8). We have looked at our major metrics. Of course there are many more and it varies across industries